In [ ]:
from IPython.display import HTML, display
import torch
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
pip -q install -U ../

In [ ]:

pip show iesta

In [ ]:
from iesta.llms.generate import Generator
import argparse
from dotenv import load_dotenv, find_dotenv

import iesta
import iesta.llms
import iesta.llms.models
from iesta.llms.models import LlamaV2, ChatGpt



In [ ]:
%pip -q install -r ../requirements.txt

In [ ]:

ineffective_args ={}
ineffective_args["liberal"] = get_data("liberal")
ineffective_args["conservative"] = get_data("conservative")


ineffective_args["liberal"].to_csv(f"../data/test_liberal_ineffective.csv")
ineffective_args["conservative"].to_csv(f"../data/test_conservative_ineffective.csv")

In [ ]:

'''
@inproceedings{hartvigsen2022toxigen,
  title={ToxiGen: A Large-Scale Machine-Generated Dataset for Implicit and Adversarial Hate Speech Detection},
  author={Hartvigsen, Thomas and Gabriel, Saadia and Palangi, Hamid and Sap, Maarten and Ray, Dipankar and Kamar, Ece},
  booktitle={Proceedings of the 60th Annual Meeting of the Association for Computational Linguistics},
  year={2022}
}'''





#df_ = pd.DataFrame([{"idx": 11, "text": "This is shit"}, {"idx": 333, "text": "Okay!"}, {"idx": 1, "text": "Arabs are really really really really horrible"}])
#toxicity_scores = pd.DataFrame(score_toxicity(df_["text"].values.tolist()))
#df_["TOXIC"] = toxicity_scores["toxic"]

In [228]:
all_experiments = _fetch_all_experiments()
exceptions_dict = {}# {k:[] for k in all_experiments}

exceptions_dict["liberal_chatgpt_0shot"] = [44708, 35204, 3176, 42440, 34248, 51598, 45614, 38169, 16602, 3259, 252,]
exceptions_dict["conservative_chatgpt_0shot"] = [79903, 17012, 66698, 824, 43599, 46682, 78779, 78643, 58514, 43596, 25775, 32238, 61381]
exceptions_dict["liberal_chatgpt_1shot"] = [252, 20310, 18397, 9161, 45614]
exceptions_dict["conservative_chatgpt_1shot"] = [79903, 66698, 40673, 23110, 13417, 46682, 78778, 78643, 11374, 43596, 9181, 49307, 48861,] # 10968 "Vote for me for bacon and strippers"

total = 0
for k,v in exceptions_dict.items():
    print(f"{k}; {len(v)}")
    total = total+ len(v)

total
total*100/14000

liberal_chatgpt_0shot; 11
conservative_chatgpt_0shot; 13
liberal_chatgpt_1shot; 5
conservative_chatgpt_1shot; 13


0.3

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter

def get_n_grams(col, n=3):
    # Tokenize the text data
    text_data = col
    tokens = []
    for sentence in text_data:
        sentence = re.sub('[^a-zA-Z0-9]+', ' ', sentence).lower()
        sentence = sentence.replace("and", "")
        sentence = sentence.replace("that", "")
        sentence = sentence.replace("to", "")

        tokens += word_tokenize(sentence)

    # Generate all possible 3-grams
    n_grams = []
    for i in range(len(tokens)-(n-1)):
        n_grams.append(tuple(tokens[i:i+n]))

    # Count the frequency of each 3-gram
    frequencies = Counter(n_grams)

    # Sort the 3-grams by frequency
    top_n_grams = sorted(frequencies.items(), key=lambda x: x[1])[-30:]

    # Print or store the results
    print(top_n_grams)


#get_n_grams(error_logs_df[error_logs_df["success"] == False ]["llm_out_msge"], n=30)


In [229]:
from glob import glob
processed_exps = glob("../data/llms_out/new/processed/*.csv")
processed_toxicity_dict= {}

def _add_toxic_lbl(row):
    row["toxic_str"] = "Toxic" if row["toxic"] >= 0.5 else "Not Toxic"
    row["success_int"] = 1 if row["success"] == True else 0
    return row

for p in processed_exps: 
    experiment = p.split("\\")[-1].replace(".jsonl", "")
    ideology = experiment.split("_")[0]
    processed_df = pd.read_csv(p)
    print(f"len before: {len(processed_df)}")
    processed_df = pd.merge(processed_df, ineffective_args[ideology][keep_ori_cols], how="inner", on="idx")
    print(f"len after: {len(processed_df)}\n ++++")
    if len(processed_df) < 3500: print(experiment)
    processed_df= processed_df.apply(_add_toxic_lbl, axis=1,)
    processed_toxicity_dict[experiment] = processed_df
    break

len before: 3563
len after: 3402
 ++++
conservative_chatgpt_0shot_processed.csv


In [ ]:

for experiment in experiments:
    print(f"\n*********************************** {experiment} ***********************************")
    processed = process_llm_generated_args(experiment)
    print(pd.crosstab(processed["prompt"], processed["type"]))

    print(pd.crosstab(processed["prompt"], processed["success"]))

In [ ]:
processed_toxicity_dict.keys()

In [ ]:
# Import seaborn
import seaborn as sns

# Apply the default theme
sns.set_theme()

In [ ]:
titanic = sns.load_dataset("titanic")
titanic.head()

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

df = processed_toxicity_dict['liberal_llamav2_0shot_steered_medianl0.2_processed.csv']
# sns.countplot(data=df, x="success", hue="toxic_str")
counter_arr = []
for k, experiment in processed_toxicity_dict.items():
    # exp_res = {"experiment": experiment}
    
    exp_res =experiment[["success", "toxic_str"]].groupby(["success", "toxic_str"])["toxic_str"].agg([ "count"]).add_prefix('toxicity_').reset_index()
    exp_res["experiment"] = k.replace(".csv", "")
    exp_res["ideology"] = k.split("_")[0]
    exp_res["model"] = k.split("_")[1]
    exp_res["Shot"] = k.split("_")[2]
    
    counter_arr.append(exp_res)


def _apply_format(row):
    row["success_toxic"] = f"{'Response' if row['success'] else 'No Response'} w {row['toxic_str'].replace('Not ', 'Non-')} Argument"
    return row
concat_count_df = pd.concat(counter_arr, ignore_index=True, )
concat_count_df = concat_count_df.apply(_apply_format, axis=1)
concat_count_df = concat_count_df[["success_toxic", "toxicity_count", "experiment", "model"]]
concat_count_df.head()


In [ ]:
#concat_count_df.set_index("experiment")
ax = sns.barplot(concat_count_df, x="experiment", y="toxicity_count", hue="success_toxic", dodge=True)#, col="model")
#ax.plot(1955, 3600, "*", markersize=10, color="r")


In [201]:
experiment

,Unnamed: 0,idx,prompt,ineffective_argument,generated,effective_argument,cannot_paraphrase,type,flag,success,incontent_ideology_mentioned,ideology_mentioned,exception,llm_out_msge,category,round,debate_id,toxic,toxic_str,success_int
0,0,28489,base,Trolls DO serve a valuable function in society...,"""Trolls play a crucial role in society by effe...",Trolls play a crucial role in society by effec...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Funny,2,Internet-trolls-serve-a-valuable-function-in-s...,0.048330,Not Toxic,1
1,1,28489,content,Trolls DO serve a valuable function in society...,"""Trolls play a crucial role in society by effe...",Trolls play a crucial role in society by effec...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Funny,2,Internet-trolls-serve-a-valuable-function-in-s...,0.048330,Not Toxic,1
2,2,28489,content_style,Trolls DO serve a valuable function in society...,"""Trolls play a crucial role in society by effe...",Trolls play a crucial role in society by effec...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Funny,2,Internet-trolls-serve-a-valuable-function-in-s...,0.048330,Not Toxic,1
3,3,28489,content_style_ideology,Trolls DO serve a valuable function in society...,"""Trolls play a crucial role in society by effe...",Trolls play a crucial role in society by effec...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Funny,2,Internet-trolls-serve-a-valuable-function-in-s...,0.048330,Not Toxic,1
4,4,28489,style,Trolls DO serve a valuable function in society...,"""Internet trolls play a crucial role in societ...",Internet trolls play a crucial role in society...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Funny,2,Internet-trolls-serve-a-valuable-function-in-s...,0.048330,Not Toxic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397,3404,82002,content_style,I believe vampires are deffently the best beca...,"""I firmly believe that vampires are unequivoca...",I firmly believe that vampires are unequivocal...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Philosophy,0,which-is-better-vampires-or-werewolves/1/,0.354186,Not Toxic,1
3398,3405,82002,content_style_ideology,I believe vampires are deffently the best beca...,"""I firmly believe that vampires are undoubtedl...",I firmly believe that vampires are undoubtedly...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Philosophy,0,which-is-better-vampires-or-werewolves/1/,0.354186,Not Toxic,1
3399,3406,82002,style,I believe vampires are deffently the best beca...,"""I firmly believe that vampires are unequivoca...",I firmly believe that vampires are unequivocal...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Philosophy,0,which-is-better-vampires-or-werewolves/1/,0.354186,Not Toxic,1
3400,3407,82002,style_ideology,I believe vampires are deffently the best beca...,"""I firmly believe that vampires are undoubtedl...",I firmly believe that vampires are undoubtedly...,False,between quotations,False,True,False,False,generated equal effective,"""-effective_argument-""",Philosophy,0,which-is-better-vampires-or-werewolves/1/,0.354186,Not Toxic,1


In [217]:
import numpy as np
from scipy.stats import pointbiserialr

for k, main_df in processed_toxicity_dict.items():
    # exp_res = {"experiment": experiment}
    if k.split("_")[1] != "chatgpt":
        for prompt, experiment in main_df.groupby("prompt"):
        # sample data
        
            success_arr = [(1 if x==True else 0) for x in experiment["success"].values.tolist()]
            
            binary_categorical_data = np.array(success_arr)
            numeric_data = np.array(experiment["toxic"].values.tolist()) # Must be coded as 0s and 1s

            corr, p_value = pointbiserialr(numeric_data, binary_categorical_data)

            print(f'{k}-{prompt}: {round(corr, 2):.2f}, {p_value:.6f} {p_value < 0.01}')

    

conservative_llamav2_0shot_processed.csv-base: -0.16, 0.000357 True
conservative_llamav2_0shot_processed.csv-content: -0.17, 0.000104 True
conservative_llamav2_0shot_processed.csv-content_style: -0.21, 0.000003 True
conservative_llamav2_0shot_processed.csv-content_style_ideology: -0.21, 0.000004 True
conservative_llamav2_0shot_processed.csv-ideology: -0.21, 0.000002 True
conservative_llamav2_0shot_processed.csv-style: -0.21, 0.000002 True
conservative_llamav2_0shot_processed.csv-style_ideology: -0.05, 0.251746 False
conservative_llamav2_0shot_steered_meanl0.2_processed.csv-base: -0.24, 0.000000 True
conservative_llamav2_0shot_steered_meanl0.2_processed.csv-content: -0.21, 0.000004 True
conservative_llamav2_0shot_steered_meanl0.2_processed.csv-content_style: -0.26, 0.000000 True
conservative_llamav2_0shot_steered_meanl0.2_processed.csv-content_style_ideology: -0.11, 0.014776 False
conservative_llamav2_0shot_steered_meanl0.2_processed.csv-ideology: -0.21, 0.000003 True
conservative_llama